In [1]:
import pandas as pd
import numpy as np

# Potential Outcomes, Ignorability, and Treatment Effects

We generally care about some causal state, $D$, and its effect on some outcome $Y$. We'd like to make an informed choice about an action we might take, and choose and enforce a value for $D$ that reaches our objective for the value of $Y$. We might prescribe medicine, $D=1$, if we think it increases the chances of recovery, $Y=1$, from a disease.

We can define potential outcomes, $Y^0$ and $Y^1$, as the outcomes someone might have if they don't take the medicine, $D=0$, or if they do, $D=1$. If someone takes the medicine, $D=1$, then we observe $Y = Y^1$. If they don't, then we observe $Y=Y^0$ for that person.

If we assume these potential outcomes are both defined, and that they are both statistically independent of $D$, then we say that $D$ is _ignorable_ for the outcome, $Y$. We can write this as $D \perp \{Y^0, Y^1\}$.

If we run a randomized, controlled experiment, we are forcing $D$ to be ignorable. We could do this by flipping a coin to choose someone's treatment status. Whether someone would recover if they took the medicine, $Y^1$, isn't determined by whether they take the medicine. The same is true for $Y^0$. The assignment, $D$, to one state or the other just selects which of $Y^1$ and $Y^0$ are actually realized. We force $D$ to be independent of all other factors by choosing it by coin toss. The only things it can be dependent with are the things that it causes.

In general, we can't observe $Y^1$ and $Y^0$ simultaneously for any individual. That poses a problem when we'd like to judge the effectiveness of a medicine. We'd like to know how much more likely someone is to recover if they had taken the medicine than if they hadn't. That would be the average difference $\delta = E[Y^1 - Y^0]$.  Unfortunately, we can't measure this difference directly. The is known as the _fundamental problem of causal inference_. 

We can use the linearity of expectation to try to get somewhere, and write this as $\delta = E[Y^1] - E[Y^0]$. These are two population-level averages, so they still require knowing $Y^1$ and $Y^0$ over the whole population. What happens if the people in the $D=0$ state are different in some way than the people in the $D=1$ state? Then $E[Y^1]$ or $E[Y^0]$ might be different in each of these subpopulations, and it would be reasonable to expect the population averages $E[Y^1]$, $E[Y^0]$ are different from the group averages, e.g. $E[Y^1|D=1]$ and $E[Y^0|D=0]$.

It looks futile to estimate $\delta$, but we can actually do it if we're willing to make some assumptions about statistical dependences. 

If we assume ignorability, then the implication is that the population averages, $E[Y^1]$ and $E[Y^0]$, are independent of $D$. Then, we can write 

$$\delta = E[Y^1] - E[Y^0] = E[Y^1|D=1] - E[Y^0|D=0]$$. 

Then, remembering that $Y^1$ is the value of $Y$ we observe in the subpopulation where $D=1$ (and likewise for $Y^0$ and $D=0$), we can write 

$$\delta = E[Y^1] - E[Y^0] = E[Y|D=1] - E[Y|D=0]$$.

This has let us get rid of the (unobserved) potential outcomes in favor of the observed outcomes, $Y$. Remember, the critical assumption here was that $D$ is _ignorable_, or that we effectively have a randomized, controlled experiment.

## Generating Data with Ignorability

We can simulate one by generating some data like


In [2]:
N = 1000  # population size

d = np.random.binomial(1, p=0.5, size=N)  # flip a coin to assign treatment
y0 = np.random.normal(0, size=N)
y1 = np.random.normal(1, size=N)

y = (d==1)*y1 + (d==0)*y0

df = pd.DataFrame({'D': d, 'Y': y})

Here, we use lowercase $d$ to indicate the value of the random variable $D$, and lowercase $y$ to indicate the value of the variable $Y$. We'll let $Y$ be real-valued to indicate some measure of health, where higher values are better, and lower values are worse.

Notice the potential outcomes are independent of $D$, and that $D$ only serves to select which potential outcome $Y$ takes. The result is that even though $Y^0$ and $Y^1$ are statistically independent of $D$, $Y$ is not. We'll see a between-group difference in the expected value of $Y$ between the test group, $D=1$, and the control group, $D=0$.

In [3]:
df.groupby('D').mean()

,Y
D,
0,0.018465
1,0.935554


We simulated the data ourselves, so we can observe the treatment effect directly as $E[Y^1 - Y^0]$,

In [4]:
(y1 - y0).mean()

0.9821923901468561

## Generating Data without Ignorability

It'll be useful to see what we mean when we don't have ignorability. Instead, $D$ is determined by other factors in the world, instead of our intervention. We can simulate those other factors. We'll call them $Z$.

In [5]:
z = np.random.uniform(0, 1,size=N)

d = np.random.binomial(1, p=z)

y0 = np.random.normal(0, size=N)
y1 = np.random.normal(1, size=N)

y = (d==1)*y1 + (d==0)*y0

df = pd.DataFrame({'D': d, 'Y': y, 'Z': z, 'Y0': y0, 'Y1': y1})

Notice that we don't have ignorability yet! We can see (since we're generating the data ourselves, with explict potential outcomes) that there's still independence between the potential outcomes and $D$.

In [6]:
df[['D', 'Y0', 'Y1']].corr()

,D,Y0,Y1
D,1.000000,0.000175,0.008407
Y0,0.000175,1.000000,-0.005744
Y1,0.008407,-0.005744,1.000000


To lose ignorability, we need to induce statistical dependence between $D$ and the potential outcomes. We can do that by making both depend on $Z$.

In [7]:
z = np.random.uniform(0, 1,size=N)

d = np.random.binomial(1, p=z)

y0 = np.random.normal(-z, size=N)
y1 = np.random.normal(z, size=N)

y = (d==1)*y1 + (d==0)*y0

df = pd.DataFrame({'D': d, 'Y': y, 'Z': z, 'Y0': y0, 'Y1': y1})

In [16]:
df[['D', 'Y0', 'Y1']].corr()

,D,Y0,Y1
D,1.000000,-0.126324,0.135279
Y0,-0.126324,1.000000,-0.089348
Y1,0.135279,-0.089348,1.000000


Now, people who are more likely to take the treatment (high $Z$) are less likely to recover when they don't take the treatment ($Y^0$ decreases like $-Z$). Now there's a difference in baseline health for the control state, $Y^0$, in people who normally would take the treatment (higher $Z$), compared with people who normally wouldn't (lower $Z$). There's also a difference in the effectiveness of the treatment between these groups. The higher the $Z$, the higher the treatment effect. You can see from the data-generating process that the treatment effect at some level of $Z=z$ is equal to $2z$.

## ATE, ATC, and ATT

So far, we've just looked at the average difference in potential outcomes, or the _average treatment effect_. We interpret this as the average effect of the treatment over the whole population.

Instead, we could ask more focused questions, like what is the treatment effect on people who were observed to take the treatment, $D=1$, or people who were observed not to, $D=0$. We'll call these the _average treatment effect on the treated_, or _ATT_, and the _average treatment effect on the control_, or ATC, respectively.

The ATT is defined as $\delta_{ATT} = E[Y^1 - Y^0|D=1]$, and the ATC is defined as $\delta_{ATC} = E[Y^1 - Y^0|D=0]$. Notice that if we had ignorability, you could immediately remove the dependence on $D$ in both of these definitions, and you have that ATT = ATC = ATE. These only differ when we don't have ignorability.

More generally, we could condition the treatment effect on anything to ask about the treatment effect within subpopulations. For example, in the code above, we talked about the treatment effect at different levels of $Z$, which we read off to be $\delta_{Z} = E[Y^1 - Y^0|Z=z] = 2z$. These are all generally called _conditional average treatment effects_, or CATEs.

## Relaxing Ignorability

We can relax the assumption that we have ignorability and see if we're still able to identify treatment effects. Let's look at two different alternative assumptions:

$$E[Y^1|D=1] = E[Y^1|D=0]$$

and 

$$E[Y^0|D=0] = E[Y^0|D=1]$$.

Each of these assumptions are saying that just one of the potential outcomes, either $Y^1$ or $Y^0$, is independent of $D$ in expectation. If we write out the definition of the conditional expectations, we'll find that this lets us identify some CATEs when we can't necessarily identify the ATE.

First, the ATT is

$$E[Y^1 - Y^0|D=1] = E[Y^1|D=1] - E[Y^0|D=1]$$

The last term on the right is counterfactual: we don't observe $Y^0$ in the $D=1$ population. We can use the second assumption to get rid of it in favor of observed quantities! Plugging in the second assumption, we get

$$E[Y^1 - Y^0|D=1] = E[Y^1|D=1] - E[Y^0|D=0]$$

and now everything is measureable. You can do the same procedure for the ATC, using the first assumption.